In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# datapath = "data/"
# all_embeddings = []
# all_parametric = []
# for i in trange(2048):
#     dir = datapath + f"batch_{i}/"
#     embeddings = pd.read_csv(dir + "embeddings.csv", index_col=0)
#     parametric = pd.read_csv(dir + "parametric.csv", index_col=0)
#     parametric = parametric.loc[embeddings.index,:]
#     all_embeddings.append(embeddings)
#     all_parametric.append(parametric)
# all_embeddings = pd.concat(all_embeddings)
# all_parametric = pd.concat(all_parametric)
# all_embeddings.to_csv("all_embeddings.csv")
# all_parametric.to_csv("all_parametric.csv")

all_embeddings = pd.read_csv("all_embeddings.csv", index_col=0)
all_parametric = pd.read_csv("all_parametric.csv", index_col=0)

In [3]:
x_train, x_test, y_train, y_test = train_test_split(all_parametric, all_embeddings, test_size=0.05, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.05, random_state=42)
x_train = torch.tensor(x_train.values, dtype=torch.float32)
x_val = torch.tensor(x_val.values, dtype=torch.float32)
x_test = torch.tensor(x_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x_train = x_train.to(device)
x_val = x_val.to(device)
x_test = x_test.to(device)
y_train = y_train.to(device)
y_val = y_val.to(device)
y_test = y_test.to(device)

In [89]:
# Residual Block class
class ResidualBlock(nn.Module):
    def __init__(self, input_size, layer_size, num_layers):
        super(ResidualBlock, self).__init__()
        self.layers = self._make_layers(input_size, layer_size, num_layers)

    def _make_layers(self, input_size, layer_size, num_layers):
        layers = []
        layers.append(nn.Linear(input_size, layer_size))
        layers.append(nn.ReLU())
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(layer_size, layer_size))
            layers.append(nn.ReLU())
        layers.append(nn.BatchNorm1d(layer_size))
        return nn.Sequential(*layers)

    def forward(self, x):
        residual = x
        out = self.layers(x)
        total = out + residual
        return total


# Residual Network class
class ResidualNetwork(nn.Module):
    def __init__(self, input_size, output_size, layer_size, layers_per_block, num_blocks):
        super(ResidualNetwork, self).__init__()
        self.initial_layer = nn.Linear(input_size, layer_size)
        self.blocks = self._make_blocks(layer_size, layers_per_block, num_blocks)
        self.final_layer = nn.Linear(layer_size, output_size)

    def _make_blocks(self, layer_size, layers_per_block, num_blocks):
        blocks = []
        for _ in range(num_blocks):
            blocks.append(ResidualBlock(layer_size, layer_size, layers_per_block))
        return nn.Sequential(*blocks)

    def forward(self, x):
        out = self.initial_layer(x)
        out = self.blocks(out)
        out = self.final_layer(out)
        return out


# Example usage
input_size = len(all_parametric.columns)
output_size = len(all_embeddings.columns)
layer_size = 128
layers_per_block = 2
num_blocks = 3

model = ResidualNetwork(input_size, output_size, layer_size, layers_per_block, num_blocks).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [90]:
patience = 5
best_val_loss = float('inf')
counter = 0
num_epochs = 100
batch_size = 16

with torch.autograd.set_detect_anomaly(True):
    for epoch in range(num_epochs):
        # Shuffle data indices
        indices = np.arange(len(x_train))
        np.random.shuffle(indices)

        model.train()
        for i in range(0, len(x_train), batch_size):
            batch_indices = indices[i:i + batch_size]
            inputs = x_train[batch_indices]
            targets = y_train[batch_indices]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(x_val)
            val_loss = criterion(val_outputs, y_val)

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch}.")
                break

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Validation Loss: {val_loss.item()}")

Epoch [1/100] - Validation Loss: 0.013147275894880295
Epoch [2/100] - Validation Loss: 0.009379365481436253
Epoch [3/100] - Validation Loss: 0.007882894948124886
Epoch [4/100] - Validation Loss: 0.011997319757938385
Epoch [5/100] - Validation Loss: 0.00642175180837512
Epoch [6/100] - Validation Loss: 0.005273997317999601
Epoch [7/100] - Validation Loss: 0.004541254602372646
Epoch [8/100] - Validation Loss: 0.22118894755840302
Epoch [9/100] - Validation Loss: 0.004190958105027676
Epoch [10/100] - Validation Loss: 0.00399976409971714
Epoch [11/100] - Validation Loss: 0.0038859383203089237
Epoch [12/100] - Validation Loss: 0.0042318375781178474
Epoch [13/100] - Validation Loss: 0.004141479730606079
Epoch [14/100] - Validation Loss: 0.0038009637501090765
Epoch [15/100] - Validation Loss: 0.003786378540098667
Epoch [16/100] - Validation Loss: 0.003919878043234348
Epoch [17/100] - Validation Loss: 0.003854452632367611
Epoch [18/100] - Validation Loss: 0.0038776900619268417
Epoch [19/100] - V

In [91]:
targets = y_test.clone().cpu()

In [92]:
preds = model(x_test).cpu()

In [93]:
a_norm = preds / preds.norm(dim=1)[:, None]
b_norm = targets / targets.norm(dim=1)[:, None]
res = torch.mm(a_norm, b_norm.transpose(0,1))
print(res)

tensor([[0.9937, 0.9568, 0.9563,  ..., 0.9543, 0.9605, 0.9163],
        [0.9635, 0.9908, 0.9492,  ..., 0.9638, 0.9517, 0.9356],
        [0.9586, 0.9288, 0.9933,  ..., 0.9589, 0.9476, 0.9551],
        ...,
        [0.9567, 0.9531, 0.9701,  ..., 0.9887, 0.9687, 0.9459],
        [0.9692, 0.9420, 0.9676,  ..., 0.9724, 0.9924, 0.9361],
        [0.9369, 0.9330, 0.9671,  ..., 0.9406, 0.9340, 0.9886]],
       grad_fn=<MmBackward0>)


In [94]:
d = torch.diag(res)

In [95]:
d_full = d.unsqueeze(dim=1).repeat(1, len(res))

In [96]:
print(torch.sum(d_full<res)/len(res)/(len(res)-1))

tensor(0.0010)


In [97]:
print(torch.sum(d_full.T<res)/len(res)/(len(res)-1))

tensor(0.0005)


In [98]:
# torch.save(model, "resnet_0010_0005.pt")